In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

Current working directory was changed to: C:\Users\viniciusbarros\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 17/01/23 - Horário: 14:30:36.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:22.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:46.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 17/01/23 - Horário: 14:36:43.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 17/01/23 - Horário: 14:36:54.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 17/01/23 - Horário: 14:37:38.  Portanto não foi baixado novamente.
vrapeelo

In [3]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
ide_data_cols = ['DthEnvio','IdeUsinaOutorga']
ide_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Baixa bancos de dados

In [4]:
# Carrega BD vmonitoramentoleilao => monitoramentoleilao
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao.sample(5)

,IdeUsinaOutorga,CodLeilao,DatInicioSuprimento
547,35113,03/2016,2020-03-01
1146,40567,04/2019,2025-01-01
52,28818,04/2019,2025-01-01
982,37898,04/2019,2025-01-01
408,34160,04/2017,2021-01-01


In [5]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DthEnvio' : 'Data_Envio',
        'NomUsina' : 'usina_nome',
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma.sample(5)

,IdeUsinaOutorga,Data_Envio,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV
10314,32843,2021-04-05 19:09:43.873,2020-06-15,NaT,2020-11-01,NaT,NaT,2020-06-20,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
24493,49405,2022-05-02 10:29:38.907,NaT,2024-12-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
16638,35249,2021-10-05 19:11:44.120,2021-04-05,NaT,2021-03-31,NaT,NaT,2021-06-30,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
16402,40871,2021-10-05 15:33:50.963,NaT,2022-01-10,NaT,NaT,NaT,2021-04-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
26727,49398,2022-06-03 16:15:18.367,NaT,2022-07-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [6]:
# Carrega BD vrapeelcontratorecurso => mercado

cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)

# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

# Cria dataframe input_rapeel

In [7]:
# Carrega BD vrapeelacesso => acesso_contratos
input_rapeel = pd.merge(mercado,ambiental_datas,on=ide_data_cols,how="inner")
input_rapeel = pd.merge(input_rapeel,acesso_contratos,on=ide_data_cols,how="inner")
input_rapeel = input_rapeel.loc[input_rapeel.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
input_rapeel.sample(5)

,DthEnvio,IdeUsinaOutorga,DatConclusaoACL,DatValidadeLI,DatValidadeLP,DatValidadeLO,NomSitContratoCCD,NomSitContratoCUST,NomSituacaoContratoCUSD,NomSitContratoCCT
139,2021-03-05 17:00:18.470,37077,2019-09-25,2024-03-04,2019-08-30,NaT,Não se Aplica,Válido,Não se Aplica,Válido
2622,2023-01-05 19:57:18.513,49434,NaT,2031-09-29,2031-09-29,NaT,Não se Aplica,Válido,Não se Aplica,Não Assinado
2129,2023-01-04 21:27:55.483,34781,2022-03-29,NaT,2025-01-14,NaT,Não se Aplica,Válido,Não se Aplica,Não Assinado
540,2022-12-04 21:24:43.027,34312,NaT,NaT,2021-08-28,NaT,Não se Aplica,Não Assinado,Não se Aplica,Não Assinado
1961,2023-01-05 17:51:57.917,29173,NaT,2023-12-23,2023-12-23,NaT,Não Assinado,Não se Aplica,Não Assinado,Não se Aplica


In [8]:
definicoes =   input_rapeel #pd.merge(input_rapeel,ug_rapeel,on=ide_data_cols,how='inner')
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]
definicoes[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
646,57201,2032-01-11,2032-01-11,2032-01-11,2032-01-11
430,43215,2024-07-07,2021-04-23,2021-05-15,2024-07-07
449,50746,2026-05-06,2026-05-06,2032-12-31,2026-05-06
1377,52258,NaT,NaT,2023-11-26,2023-11-26
726,46510,NaT,2025-06-02,2022-02-16,2025-06-02


In [9]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]

monitoramentodados = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]

In [10]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao
rename_cols = {'DatPrevistaComercial':'DatPrevistaComercial'}
definicoes_monitoramento = pd.merge(monitoramentodados,definicoes,on='IdeUsinaOutorga',how="left").rename(columns=rename_cols).drop_duplicates()
definicoes_monitoramento = definicoes_monitoramento.loc[definicoes_monitoramento.IdcUsinaMonitorada == "Sim"]

definicoes_monitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoes_monitoramento.loc[definicoes_monitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()) & (definicoes_monitoramento.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
definicoes_monitoramento.loc[(definicoes_monitoramento.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


definicoes_monitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(definicoes_monitoramento[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

definicoes_monitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSitContratoCCD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCCT == nsa) &
    definicoes_monitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoes_monitoramento.loc[
(    ((definicoes_monitoramento.NomSitContratoCCD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCCT == valido)) &
    definicoes_monitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSitContratoCCD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCCT == vencido) |
    (definicoes_monitoramento.NomSitContratoCCD == nass) |
    (definicoes_monitoramento.NomSitContratoCCT == nass )) &
    definicoes_monitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoes_monitoramento['CondicaoUso'] = "Não informado"

definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoes_monitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoes_monitoramento.loc[
    ((definicoes_monitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoes_monitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoes_monitoramento.NomSitContratoCUST == vencido) |
    (definicoes_monitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoes_monitoramento.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoes_monitoramento["PPA"] = "Ambos"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoes_monitoramento.loc[
    (((definicoes_monitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoes_monitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoes_monitoramento[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
1,NaT,2026-03-09,2002-12-11,LI,2026-03-09
2,NaT,NaT,NaT,Sem LP,NaT
3,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
7,NaT,NaT,NaT,Não Informado,NaT
8,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
3093,NaT,NaT,NaT,Não Informado,NaT
3094,NaT,NaT,NaT,Não Informado,NaT
3095,NaT,NaT,NaT,Não Informado,NaT
3096,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
2676,51983,Não se Aplica,Não Assinado,2023-01-02 07:27:47.543,Sem Conexão
1269,43148,Não se Aplica,Válido,2023-01-03 14:47:03.840,OK
464,34515,Não se Aplica,Não Assinado,2023-01-05 12:34:49.990,Sem Conexão
1383,44465,Não se Aplica,Válido,2023-01-04 17:43:18.157,OK
1780,47312,Não se Aplica,Válido,2023-01-05 18:34:44.647,OK
1195,40758,Não se Aplica,Não Assinado,2023-01-05 17:12:17.730,Sem Conexão
2096,49375,Não se Aplica,Válido,2023-01-05 18:34:05.603,OK
2959,54871,Não se Aplica,Não Assinado,2023-01-05 19:25:52.347,Sem Conexão
2270,49694,Não se Aplica,Válido,2023-01-02 18:09:33.830,OK
1059,38133,Não se Aplica,Não Assinado,2023-01-03 17:58:12.393,Sem Conexão


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1682,47049,Não se Aplica,Não Assinado,Sem Conexão
1338,44365,Não se Aplica,Válido,OK
1845,47394,Não se Aplica,Vencido,OK
1614,46587,Válido,Não se Aplica,OK
2960,54872,Não se Aplica,Não Assinado,Sem Conexão
2293,49818,Não se Aplica,Válido,Sem Conexão
184,32076,Não se Aplica,Não Assinado,Sem Conexão
1106,38348,Não Assinado,Não se Aplica,Sem Conexão
2330,49890,Não se Aplica,Não Assinado,Sem Conexão
2622,51646,Não se Aplica,Não Assinado,Sem Conexão


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
2948,54844,Fora do ACR,NaT,Nenhum
2017,48913,Fora do ACR,NaT,Nenhum
1968,48611,Fora do ACR,NaT,Nenhum
790,37494,Fora do ACR,2021-05-03,ACL
1910,47483,Fora do ACR,NaT,Nenhum
1637,46782,Fora do ACR,NaT,Nenhum
1887,47460,Fora do ACR,NaT,Nenhum
2711,52135,Fora do ACR,NaT,Nenhum
1825,47371,Fora do ACR,NaT,Nenhum
2389,50042,Fora do ACR,NaT,Nenhum


In [11]:
# Apenas mudança de nome
tabela_criterio = definicoes_monitoramento

In [12]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabela_criterio['criterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabela_criterio['dscriterio'] = np.select(
    [
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
2594,51568,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2062,49197,Não Iniciada,Nenhum,Sem LP,OK,6,Sem LP
2732,52283,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2669,51867,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2291,49816,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1277,43156,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1627,46734,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1349,44416,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1828,47374,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
414,34282,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [13]:
slp = "Sem LP"
sim = "Sim"
tabela_criterio['criterio_novo'] = np.select(
    [
        (tabela_criterio.IdcSemPrevisao == sim),
        (tabela_criterio.IdcSituacaoObra == ea),
        (( tabela_criterio.IdcSituacaoObra == na)  &  (tabela_criterio.PPA != nenhum) ),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso == ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabela_criterio.CondicaoUso != ok)),
        ((tabela_criterio.IdcSituacaoObra == na) & (tabela_criterio.PPA == nenhum) & (tabela_criterio.CondicaoAmbiental == "LP") & (tabela_criterio.CondicaoUso != ok)),
        (tabela_criterio.IdcSituacaoObra == paralisada),
        (tabela_criterio.CondicaoAmbiental == "Sem LP"),
        ((tabela_criterio.DthEnvio.isna()) & (tabela_criterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabela_criterio[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
608,35636,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1958,48596,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
711,36983,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
1017,38057,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2012,48908,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"


In [14]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100


In [15]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento ']:
    if 'SigTipoGeracao' in calculo_previsao.columns:
        calculo_previsao.drop(['SigTipoGeracao'],inplace=True,axis=1)

### criterios_da_previsao

In [16]:
criterios_da_previsao  = pd.merge(
                                    tabela_criterio,calculo_previsao,
                                    on="IdeUsinaOutorga",how='inner'
                                    )
criterios_da_previsao = pd.merge(criterios_da_previsao,ug_rapeel,left_on= ['IdeUsinaOutorga','NumUgUsina'],right_on=['IdeUsinaOutorga','NumOperacaoUg'],how='left')

del criterios_da_previsao['NumOperacaoUg']

# Regras

In [17]:
################################ regra_antiga ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regra_antiga'] = criterios_da_previsao[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regra_antiga'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] >= 3) & ( (criterios_da_previsao['IdcObraParalisada'] == "Sim")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[criterios_da_previsao['criterio'] >= 3 & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regra_antiga'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'regrapmo'] = criterios_da_previsao['Previsao_OC']
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8) ),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(3*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(4*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'regrapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')
criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5) | (criterios_da_previsao['criterio'] == 7),'regrapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'regrapmo'] = pd.NA

In [18]:
################################ dscjustificativapmo ################################
list_condicoes = [
    (criterios_da_previsao['IdcSemPrevisao'] == "Sim"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"), #& ( criterios_da_previsao['Previsao_OC'] > criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"), #& ( criterios_da_previsao['Previsao_OC'] < criterios_da_previsao['DatPrevistaComercial']),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 8)) & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Dat_OC_obrigacao']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")), # & ( criterios_da_previsao['DatPrevistaComercial'] > criterios_da_previsao['Previsao_OC']),
    ( (criterios_da_previsao['criterio'] == 1) | (criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")), # & ( criterios_da_previsao['DatPrevistaComercial'] < criterios_da_previsao['Previsao_OC']), 
    (criterios_da_previsao['criterio'] == 3),
    (criterios_da_previsao['criterio'] == 4) | (criterios_da_previsao['criterio'] == 5),
    (criterios_da_previsao['criterio'] == 7),
    (criterios_da_previsao['criterio'] == 6),
]

list_values = [
    "Analisar justificativa",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras",
    "Usina liberada para testes",
    "Testes não concluídos",
    "Paralisação de Obras",
    "Paralisação de Obras",
    "Estágio Atual das Obras",
    "Estágio Atual das Obras" ,
    "Não celebrado contrato de uso do sistema de transmissão/distribuição",
    "Situação da Licença de Instalação",
    "Sem Rapeel",
    "Sem Licenciamento Ambiental"
]

criterios_da_previsao['dscjustificativapmo'] = np.select(list_condicoes,list_values,default="Null")


################################ criterionovopmo ################################
list_condicoes = [
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT")),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT")),
    ((criterios_da_previsao['criterio_novo'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 2)),
    ((criterios_da_previsao['criterio_novo'] == 3)),
    ((criterios_da_previsao['criterio_novo'] == 5)),
    ((criterios_da_previsao['criterio_novo'] == 4)),
    ((criterios_da_previsao['criterio_novo'] == 6)),
    ((criterios_da_previsao['criterio_novo'] == 7)),
    ((criterios_da_previsao['criterio_novo'] == 8)),
    ((criterios_da_previsao['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

criterios_da_previsao['criterionovopmo'] = np.select(list_condicoes,list_values)



list_condicoes = [
    ((criterios_da_previsao['criterio_novo'] == 9)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT")),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT")),
    ((criterios_da_previsao['criterio_novo'] == 1)),
    ((criterios_da_previsao['criterio_novo'] == 2)),
    ((criterios_da_previsao['criterio_novo'] == 3)),
    ((criterios_da_previsao['criterio_novo'] == 4)),
    ((criterios_da_previsao['criterio_novo'] == 5)),
    ((criterios_da_previsao['criterio_novo'] == 7)),
    ((criterios_da_previsao['criterio_novo'] == 6)),
    ((criterios_da_previsao['criterio_novo'] == 8))
]


list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas == Previsão OC maior entre data RAPEEL e data compromisso"
]


criterios_da_previsao['dsccriterionovo'] = np.select(list_condicoes,list_values)


In [19]:
################################ regranovapmo ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'regranovapmo'] = pd.NA

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].isna()),'regranovapmo'] = criterios_da_previsao['DatPrevistaComercial']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['Previsao_OC'].notna()),'regranovapmo'] = criterios_da_previsao['Previsao_OC']

criterios_da_previsao.loc[(criterios_da_previsao['criterio_novo']== 1) | (criterios_da_previsao['criterio_novo']== 2) ,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(5*365,'D')

criterios_da_previsao.loc[((criterios_da_previsao['criterio_novo'] == 3) | (criterios_da_previsao['criterio_novo'] == 4) | (criterios_da_previsao['criterio_novo'] == 5) ),'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 6,'regranovapmo'] = pd.NA

criterios_da_previsao["Dummy"] = hoje + pd.Timedelta(6*365,'D')
criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 7,'regranovapmo'] =  criterios_da_previsao[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

criterios_da_previsao.loc[criterios_da_previsao['criterio_novo']== 8,'regranovapmo'] = criterios_da_previsao[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'criterio_pmo_ts'] = 8

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'criterio_pmo_ts'] = 0.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'criterio_pmo_ts'] = 0.2

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'criterio_pmo_ts'] = 0.3

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'criterio_pmo_ts'] = 1.1

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 1.2

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'criterio_pmo_ts'] = 2.2

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'criterio_pmo_ts'] = 2.1

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'criterio_pmo_ts'] = 3

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'criterio_pmo_ts'] = 4

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'criterio_pmo_ts'] = 5

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'criterio_pmo_ts'] = 7

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'criterio_pmo_ts'] = 6


list_condicoes = [
    (criterios_da_previsao['IdcSemPrevisao'] == "Sim"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),
    (criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),
    ((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),
    ( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),
    ((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),
    ( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),
    (criterios_da_previsao['criterio'] == 3),
    (criterios_da_previsao['criterio'] == 4),
    (criterios_da_previsao['criterio'] == 5),
    (criterios_da_previsao['criterio'] == 7),
    (criterios_da_previsao['criterio'] == 6),
]

list_values = [8,0.1,0.2,0.3,1.1,1.2,2.2,2.1,3,4,5,7,6]
criterios_da_previsao['criterio_pmo_ts'] = np.select(list_condicoes,list_values)


In [20]:
################################ dsccriterio_pmo_ts ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT") & (criterios_da_previsao['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 0) & (criterios_da_previsao['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

criterios_da_previsao.loc[((criterios_da_previsao['PPA'] != "Nenhum")) & ((criterios_da_previsao['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 1)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[((criterios_da_previsao['criterio'] == 2)) & ( (criterios_da_previsao['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[( (criterios_da_previsao['criterio'] == 8)) & ( (criterios_da_previsao['PPA'] == "Nenhum") & (criterios_da_previsao['CondicaoUso'] == "OK")) & ( (criterios_da_previsao['CondicaoAmbiental'] == "LI") | (criterios_da_previsao['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

criterios_da_previsao.loc[(criterios_da_previsao['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"

In [21]:
################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = criterios_da_previsao.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
criterios_da_previsao = criterios_da_previsao.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
criterios_da_previsao.loc[(criterios_da_previsao['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

criterios_da_previsao.loc[(criterios_da_previsao['IdcObraParalisada'] == "Sim") & (criterios_da_previsao['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

criterios_da_previsao.loc[(criterios_da_previsao.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################

list_condicoes = [
    (criterios_da_previsao['criterionovopmo'] == 9),
    (criterios_da_previsao['criterionovopmo'] == 8),
    (criterios_da_previsao['criterionovopmo'] == 0.2),
    (criterios_da_previsao['criterionovopmo'] == 0.1) | (criterios_da_previsao['criterio_pmo_ts'] == 0.3),
    (criterios_da_previsao['criterionovopmo'] == 1),
    criterios_da_previsao['criterionovopmo']==2,
    criterios_da_previsao['criterionovopmo']==3,
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 4),
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 5),
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 6),
    (criterios_da_previsao.criterio_pmo_ts_max_usina== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

criterios_da_previsao['dscjustificativaregranova'] = np.select(list_condicoes,list_values)


In [61]:
#criterios_da_previsao_sas = pd.read_excel("./WORK_Criteriosdaprevisao.xlsx")
#save_pickle(criterios_da_previsao_sas,'./criterios_da_previsao_sas.pickle')
#criterios_da_previsao_2_sas = pd.read_excel("./WORK_Criteriosdaprevisao2.xlsx")
#save_pickle(criterios_da_previsao_2_sas,'./criterios_da_previsao_2_sas.pickle')
#tabela_criterio_sas = pd.read_excel("./WORK_TABELCRITERIO.xlsx")
#save_pickle(tabela_criterio_sas,'./tabela_criterio_sas.pickle')

criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga")


criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
criterios_da_previsao["PY"] = True

In [62]:
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

In [63]:
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

In [64]:
merge = pd.merge(criterios_da_previsao,criterios_da_previsao_sas,on=['IdeUsinaOutorga','NumUgUsina'],suffixes=('_py','_sas'),how='outer')
merge['OC_diff'] = merge.Previsao_OC - merge.CalculoPrevisorOC_mx

# Teste previsão

In [65]:
merge[['IdeUsinaOutorga','NumUgUsina'] + [ 'Previsao_OC','CalculoPrevisorOC_mx','OC_diff','IdcUsinaMonitorada']].sort_values(by='OC_diff',ascending=False).head(10)

,IdeUsinaOutorga,NumUgUsina,Previsao_OC,CalculoPrevisorOC_mx,OC_diff,IdcUsinaMonitorada
138,30414,1,2025-03-18,2022-09-08,922 days,Sim
144,30612,1,2025-03-29,2023-01-23,796 days,Sim
225,31691,8,2023-09-02,2022-06-09,450 days,Sim
224,31691,5,2023-09-02,2022-06-09,450 days,Sim
229,31691,2,2023-09-02,2022-06-09,450 days,Sim
228,31691,7,2023-09-02,2022-06-09,450 days,Sim
227,31691,4,2023-09-02,2022-06-09,450 days,Sim
222,31691,3,2023-09-02,2022-06-09,450 days,Sim
226,31691,1,2023-09-02,2022-06-09,450 days,Sim
223,31691,6,2023-09-02,2022-06-09,450 days,Sim


# Testes critérios

## FASE ok

In [66]:
merge[merge.FASE_py != merge.FASE_sas].shape

(0, 189)

## criterionovopmo OK

In [67]:
# Check
merge[merge.CriterioNovoPMO != merge.criterionovopmo][['IdeUsinaOutorga','NumUgUsina'] + ['IdcSemPrevisao','flagOPTeste30dias_sas','flagOPTeste30dias_py','FASE_sas','FASE_py','criterio_novo','criterio_sas','criterio_py','CriterioNovoPMO','criterionovopmo']]#.drop_duplicates(subset='IdeUsinaOutorga')

,IdeUsinaOutorga,NumUgUsina,IdcSemPrevisao,flagOPTeste30dias_sas,flagOPTeste30dias_py,FASE_sas,FASE_py,criterio_novo,criterio_sas,criterio_py,CriterioNovoPMO,criterionovopmo


## criterio OK

In [68]:
# Check
merge[merge.criterio_sas != merge.criterio_py].shape

(0, 189)

## dscjustificativapmo

In [74]:
merge[merge.DscJustificativaPMO  != merge.dscjustificativapmo][['IdeUsinaOutorga','NumUgUsina'] +['IdcSemPrevisao','IdcObraParalisada','FASE_sas','criterio_sas','DatPrevistaComercial','Previsao_OC','CalculoPrevisorOC_mx','DscJustificativaPMO','DatPrevistaComercial','dscjustificativapmo']]

,IdeUsinaOutorga,NumUgUsina,IdcSemPrevisao,IdcObraParalisada,FASE_sas,criterio_sas,DatPrevistaComercial,Previsao_OC,CalculoPrevisorOC_mx,DscJustificativaPMO,DatPrevistaComercial,dscjustificativapmo


## dscjustificativaregranova OK

In [78]:
merge[merge.dscjustificativaregranova != merge.DscJustificativaRegranova]


,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_x,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_x,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_x,SigTipoGeracao,DscComercializacaoEnergia_py,DatInicioObraRealizado,DatMontagemRealizado,DthEnvio_x_py,DatConclusaoACL_py,DatValidadeLI_py,DatValidadeLP_py,DatValidadeLO_py,NomSitContratoCCD_py,NomSitContratoCUST_py,NomSituacaoContratoCUSD_py,NomSitContratoCCT_py,ValidadeAmbiental,CondicaoAmbiental,CondicaoConexao_py,CondicaoUso,PPA,criterio_py,dscriterio_py,criterio_novo,NomUsina,NumUgUsina,Previsao_OC,Dat_OC_obrigacao,DatMonitoramento_y,FASE_py,Ind_crono_norm_py,flagOPTeste30dias_py,DatPrevisaoIniciobra_y,DatInicioObraOutorgado_y,DthEnvio_y_py,...,DatValidadeContratoCCD,NomSitContratoCCT_sas,DatAssinaturaContratoCCT,DatValidadeContratoCCT,NomSituacaoContratoCUSD_sas,DatAssinaturaContratoCUSD,DatValidadeContratoCUSD,NomSitContratoCUST_sas,DatAssinaturaContratoCUST,DatValidadeContratoCUST,DatSolicitacaoLP,DatPrevistaEmissaoLP,DatEmissaoLP,DatValidadeLP_sas,NomSituacaoLP,DatSolicitacaoLI,DatPrevistaEmissaoLI,DatEmissaoLI,DatValidadeLI_sas,NomSituacaoLI,DatSolicitacaoLO,DatPrevistaEmissaoLO,DatEmissaoLO,DatValidadeLO_sas,NomSituacaoLO,IdcTipoEntrave_y,DscSolucao_y,DscObservacaoAcesso_y,DscObservacaomercado_y,DscObservacaoAmbiental_y,ValidadeAmbiental_y,CondicaoAmbiental_y,CondicaoConexao_sas,CondicaoUso_y,PPA_y,DscComercializacaoEnergia_sas,DscSistema_y,IdcViabilidade_y,IdcSituacaoObra_y,IdcObraParalisada_y,IdcSemPrevisão_y,criterio_sas,DatPrevistaComercialRapeelMX_y,DscJustificativaPrevisao,DscObservacaoUG_y,DatEmissaoAto_y,criterio_x0020_novo,dscriterio_sas,SAS,OC_diff


# BIU

In [ ]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga","DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada"]
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols]

In [85]:
# vInputCronograma tbm implementado,DATASREAL (LABEL="Idempreendimento" AS Idempreendimento)
cols = ['IdeUsinaOutorga',"DthEnvio","DatRealizacaoIII","DatRealizacaoV","DatRealizacaoXIV","DatRealizacaoXII","DatRealizacaoIV","DatRealizacaoIX","DatRealizacaoXI","DatRealizacaoX","DatRealizacaoVI","DatRealizacaoII","CodCeg","DatRealizacaoXIII","DatRealizacaoVII"]
rename_cols = {'DatRealizacaoIII': 'IO_real','DatRealizacaoXIV': 'Conc_Trans_REAL','DatRealizacaoXII': 'Comiss_Real','DatRealizacaoIX': 'DesvRio_real','DatRealizacaoXI': 'Ench_Real','DthEnvio': 'Data_Envio','DatRealizacaoII': 'canteiroReal','DatRealizacaoXIII': 'IOTrans_Real','DatRealizacaoVII': 'ME_Real_conc_eol'}
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates()[cols].rename(columns=rename_cols)
vcronograma["classe"] = vcronograma.CodCeg.str.slice(3,5)
vcronograma["CC_real"] = pd.to_datetime(pd.NA)
vcronograma.loc[vcronograma.classe == "PH","CC_real"] = vcronograma.DatRealizacaoIV
vcronograma.loc[vcronograma.classe == "CV","CC_real"] = vcronograma.DatRealizacaoV
vcronograma["ME_real"] = vcronograma.DatRealizacaoX
vcronograma.loc[vcronograma.classe == "CV","ME_real"] = vcronograma.DatRealizacaoVI
vcronograma = vcronograma.loc[vcronograma.groupby('IdeUsinaOutorga').Data_Envio.idxmax()]